In [6]:
from sklearn import metrics
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784')
x_data, y_data = mnist.data, mnist.target.astype(int)

In [9]:
import platform

if platform.system() == 'Darwin':
    device = 'cpu'
else:
    device = 'cuda'
print(device)

cpu


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.20, random_state = 42)
X_train, X_test = torch.Tensor(X_train.values), torch.Tensor(X_test.values)
y_train, y_test = torch.Tensor(y_train.values), torch.Tensor(y_test.values)

datas = [X_train, X_test, y_train, y_test]
for data in datas:
    data = data.to(device)
    print(data.shape)

torch.Size([56000, 784])
torch.Size([14000, 784])
torch.Size([56000])
torch.Size([14000])


In [3]:
import torch
import torch.nn.functional as F
import torch.nn as nn


class MLP(torch.nn.Module):

    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h3 = self.fc3(h2)
        return h3


In [5]:
lr = 0.01
epochs = 2000
model = MLP()
cirterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [14]:
loss_save_arr = []
for i in range(epochs):
    # train
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = cirterion(output, y_train.long())
    loss.backward()
    optimizer.step()
    loss_save_arr.append(loss.data)

    if(i % 100 == 0):
        print("====")
        print('epoch ', i)
        print('loss ', loss.data)
        _, pred = torch.max(output.data, axis = 1)
        print("train_accuracy {:0.3f}".format(float((pred == y_train).sum() / y_train.size(0))))

        # test
        model.eval()
        with torch.no_grad():
            output = model(X_test)
            _, pred = torch.max(output.data, axis = 1)
            print("test_accuracy {:0.03f}".format(float((pred == y_test).sum() / y_test.size(0))))



====
epoch  0
loss  tensor(80.0991)
train_accuracy 0.178
test_accuracy 0.168


KeyboardInterrupt: 